In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# import sys
# os.chdir('/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets')

In [1]:
import tensorflow as tf
import numpy as np
from models.main import SpectralGAN
from models.main import loss1, loss2
input_shape = (32,32,3)

In [2]:
model = SpectralGAN(input_shape)  

optimizer = tf.keras.optimizers.Adam(1-3)

model.compile(optimizer, loss=['mse', loss1(), loss2()])

ValueError: Dimension 3 in both shapes must be equal, but are 3 and 1. Shapes are [?,32,32,3] and [?,32,32,1].
	From merging shape 0 with other shapes. for '{{node tf.stack/stack}} = Pack[N=3, T=DT_FLOAT, axis=-1](Placeholder, Placeholder_1, Placeholder_2)' with input shapes: [?,32,32,3], [?,32,32,1], [?,32,32,1].

In [9]:
model.summary()

Model: "SpectralGAN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
encoder_2 (Encoder)             (None, 32, 32, 16)   71888       input_2[0][0]                    
__________________________________________________________________________________________________
generator_1 (Generator)         (None, 32, 32, 3)    71811       encoder_2[0][0]                  
__________________________________________________________________________________________________
discriminator_1 (Discriminator) (None, 32, 32, 1)    72033       input_2[0][0]                    
                                                                 generator_1[0][0]      

In [7]:

test = np.zeros((10,) + input_shape)

In [8]:
model.fit(x=test, y=test, batch_size=1 , epochs=10)

Epoch 1/10
(1, 32, 32, 3)


ValueError: in user code:

    C:\Users\BRAYANMONROY\miniconda3\envs\deeplearning\lib\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    c:\Users\BRAYANMONROY\Google Drive\ProyectoIA2\generative-spectral-datasets\models\main.py:13 call  *
        real = y_pred[1]
    C:\Users\BRAYANMONROY\miniconda3\envs\deeplearning\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    C:\Users\BRAYANMONROY\miniconda3\envs\deeplearning\lib\site-packages\tensorflow\python\ops\array_ops.py:1040 _slice_helper
        return strided_slice(
    C:\Users\BRAYANMONROY\miniconda3\envs\deeplearning\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    C:\Users\BRAYANMONROY\miniconda3\envs\deeplearning\lib\site-packages\tensorflow\python\ops\array_ops.py:1213 strided_slice
        op = gen_array_ops.strided_slice(
    C:\Users\BR